In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
%matplotlib inline

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

In [2]:
!ls /scratch/06040/tg853394/tc/output/redux/maria/

3dom  axisym  ctl  lwcrf  ncrf_36h  ncrf_60h  ncrf_96h


#### Read in stuff

In [7]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'
#############################################################
path_coor = '/scratch/06040/tg853394/tc/output/redux/maria/ctl/post/U.nc'
coor = xr.open_dataset(path_coor)
#Function####################################################
import json,pickle
def read_some_azimuth_fields(fileloc=None,fieldname=None):
    dict_name = {}
    for inx,obj in enumerate(fileloc):
        field_read = xr.open_dataset(obj)
        dict_name[fieldname[inx]] = field_read
    return dict_name
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None
#############################################################
ctrlvar_dict = read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_U_0-144hr.nc',
                                                 origin_path+'ctl/azim_V_0-144hr.nc',
                                                origin_path+'ctl/azim_T_0-144hr.nc'],
                                        fieldname=['U','V','T'])
ncrf36var_dict = read_some_azimuth_fields(fileloc=[origin_path+'lwcrf/azim_U_36-144hr.nc',
                                                   origin_path+'lwcrf/azim_V_36-144hr.nc',
                                                  origin_path+'lwcrf/azim_T_36-144hr.nc'],
                                          fieldname=['U','V','T'])

In [6]:
import os,sys
sys.path.insert(1, '/work/08350/tg876493/stampede2/python_codes/2020_TC_CRF/dev/freddy0218/tools/')
import derive_var

In [8]:
lwcrf_theta = derive_var.do_theta(ncrf36var_dict['T']['T'],coor.pres)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [04:45<00:00,  7.33s/it]


In [10]:
save_to_pickle('/work2/08350/tg876493/stampede2/python_codes/tcg_radiation/output/lwcrf_thetaall',lwcrf_theta)